In [1]:
%load_ext autoreload
%autoreload 2

# Dependencies

In [2]:
import datasets_handler
import evaluation_metrics
from zeroberto import ZeroBERTo
import gc
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder

google_colab = False

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:181: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


In [3]:
if google_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    # dataset_path = '/content/drive/Othercomputers/My MacBook Pro/Datasets/folhauol/folhauol_clean_df_articles.csv'
    # dataset_path = '/content/drive/MyDrive/Maquina de London/clean_df_articles_folhauol.csv'
    zeroshot_data_path = './'
else:
    dataset_path = None
    zeroshot_data_path = None

# Data

In [4]:
which_dataset = 'folhauol' 
max_inferences = 50000
random_state = 333
le = LabelEncoder()

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset,path=dataset_path)
classes_list = list(raw_data[class_col].unique())
le.fit(raw_data[class_col])
raw_data = evaluation_metrics.Encoder(raw_data,labelEncoder=le,columnsToEncode=[class_col])
data_to_label = raw_data.sample(max_inferences,random_state=random_state).sort_index()


# Parameters

In [14]:
zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual', # model for semantic similarity
    'setfit_model' : 'sentence-transformers/stsb-xlm-r-multilingual',# 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2' # model for contrastive learning and classification
    'dataset':which_dataset, ### which dataset we are using
    'class_col':class_col, ### which column of the dataset has the classes
    'data_col':data_col, ### which column of the dataset has the data
    'split':"zeroshot", ### fewshot (uses real training data) or zeroshot (uses previously selected training data)
    'labeling_method':zeroshot_method, ### how the first labeling should be executed
    'max_inferences':max_inferences, ### sample size from full dataset (number of records)
    'classes':classes_list, ### classes from the dataset
    'template': "Esse texto está relacionado a {}.", ### prompt that will be used for semantic search
    'random_state':random_state, ### random state for reproducibility
    'top_n': 8, ### how many best predictions from zeroshot should be recovered
    'training_examples': 8, ### how many examples per class should be used for training
    "batch_size" : 4, ### contrastive training batch size
    "num_pairs" : 10, ### number of text pairs that will be generated in contrastive learning
    "num_epochs" : 3, ### number of epochs training will be conducted
    # 'keep_body_frozen_setfit': True,     # Unfreeze head and freeze body -> head-only training
    'keep_body_frozen_setfit' : False, # Unfreeze head and body -> end-to-end training
    'label_encoder' : le, ### label encoder to use for encoding categorial data into int
    'average_sentence_embeddings':False, ### whether documents should be split by . and their embeddings be a mean of sentences embeddings
    'st_train_epochs': 10, ### sentence-transformer TSDAE fine-tuning --- do not use
    'st_train_batch_size': 20, ### sentence-transformer TSDAE fine-tuning --- do not use
}



# Model

In [15]:
model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
       

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


1
queries
3
4


In [16]:
model.average_sentence_embeddings

False

# Labeling

In [18]:
%%time
# if (model.config['split']=='zeroshot'): ###  selecting training data
    # model.runLabeling()
    # model.evaluateLabeling()
    # model.getLabelingMetrics()
    # model.saveLabelingResults(local_path=zeroshot_data_path)
model.config['exec_time'] = '2023_04_10__02_51_43' #### comment above and use this if labeling is already done

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 5.72 µs


Constrative Learning Data

In [19]:
df_train,df_test = datasets_handler.splitDataset(model.labeling_dataset,model.config) 
train_dataset,test_dataset = datasets_handler.buildDatasetDict(df_train,df_test)


/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:146: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(train_data.sum())


Unnamed: 0                                                   7666957
title              Cientistas estudam meios de acelerar os instin...
date               2015-11-042015-09-192017-04-262017-07-172015-0...
link               http://www1.folha.uol.com.br/ciencia/2015/04/1...
data               Cientistas estudam meios de acelerar os instin...
text               Cientistas estudam meios de acelerar os instin...
len                                                           321446
prediction_code                                                  528
top_probability                                            11.063209
dtype: object


# Training

In [20]:
### overrides config for looping second part of the model
# model.config['num_epochs'] = 5
# model.config['num_pairs'] = 20
# # model.config['keep_body_frozen_setfit'] = False

### build trainer for contrastive learning
model.buildTrainer(train_dataset)

### train
model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
# model.getPredictions()

model.predict(test_dataset['text'])
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese


Applying column mapping to training dataset
***** Running training *****
  Num examples = 1920
  Num epochs = 3
  Total optimization steps = 1440
  Total train batch size = 4


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1440 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1440 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1440 [00:00<?, ?it/s]

Predicting 50000 documents


array([10,  6,  8, ...,  8,  8,  7])

In [21]:
setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, test_dataset["class_code"])
print(setfit_all_metrics)

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics,None)
print(model.config)

{'weighted': [{'accuracy': 0.54862}, {'precision': 0.55511988723064}, {'recall': 0.54862}, {'f1': 0.5318512144614356}], 'macro': [{'accuracy': 0.54862}, {'precision': 0.4078972883851902}, {'recall': 0.5937672398467065}, {'f1': 0.44669122448790133}]}
metrics_setfit_2023_04_10__15_32_12.csv
config_setfit_2023_04_10__15_32_12.csv
{'similarity_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'setfit_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'dataset': 'folhauol', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'labeling_method': 'dotproduct', 'max_inferences': 50000, 'classes': ['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida'], 'template': 'Esse texto está relacionado a {}.', 'random_state': 333, 'top_n': 8, 'training_examples': 8, 'batch_size': 4, 'num_pairs': 10, 'num_epochs': 3, 'keep_bod